In [1]:
from pyfuseki import FusekiUpdate, FusekiQuery
import httpx
from rdflib import Graph
from pysolr import Solr
from api.src.schemas.authorities.personalName import PersonalName
from api.src.function.authorities.personalName.create_graph import MakePersonalGraph
from api.src.schemas.authorities.mads import Authority, EditVariant
from api.src.function.authorities.personalName.editPersonalName import EditAuthorityPersonalName
from api.src.function.authorities.personalName.variantPersonalName import editVariantPersonalName, addVariantPersonalName
from api.src.function.authorities.edit_uri import DelMads, PostMads
from api.src.function.authorities.personalName.editPersonalName import EditAuthorityPersonalName, EditFullerName, DeleteFullerName, PostFullerName


In [5]:
solr = Solr('http://localhost:8983/solr/authorities/', timeout=10)
response = solr.query(q="id:n80002329",  commit=True)
response

AttributeError: 'Solr' object has no attribute 'query'

In [6]:
response = solr.search(q="id:n80002329")


In [7]:
response.docs

[{'recordChangeDate': ['2023-04-12T16:25:08Z'],
  'id': 'n80002329',
  'label': ['Machado de Assis'],
  'date': ['1839-1908'],
  'birthDate': ['1839-06-21'],
  'birthPlace': ['Rio de Janeiro (Brazil)'],
  'deathDate': ['1908-09-29'],
  'fullerName': ['Joaquim Maria Machado'],
  'hasCloseExternalAuthority': ['http://id.worldcat.org/fast/48407',
   'http://www.wikidata.org/entity/Q311145'],
  'hasExactExternalAuthority': ['http://viaf.org/viaf/sourceID/LC%7Cn++80002329#skos:Concept'],
  'hasVariant': ['Machado de Assis, Joaquín María',
   'Mashado de Assiz, Zhoakin',
   'Assis, Machado de'],
  'hasAffiliation': ['http://id.loc.gov/authorities/names/n82059239'],
  'occupation': ['http://id.loc.gov/authorities/subjects/sh85034154',
   'http://id.loc.gov/authorities/subjects/sh85092863'],
  'type': ['PersonalName'],
  'uri': ['https://bibliokeia.com/authorities/name/n80002329'],
  '_version_': 1763003451047936000}]

In [11]:
[doc] = response.docs
hasCloseExternalAuthority = doc.get('hasCloseExternalAuthority')

In [15]:
d = {}
for i in hasCloseExternalAuthority:
    d[f'{i.split(".")[1]}'] = i

d

{'worldcat': 'http://id.worldcat.org/fast/48407',
 'wikidata': 'http://www.wikidata.org/entity/Q311145'}

In [2]:
id = 'n80002329'
obj = {
  "value": "MACHADÃO de ASSIS",
  "type": "PersonalName"
}

request = Authority(**obj)
variant = PostFullerName(id, request)
print(variant)

PREFIX name: <https://bibliokeia.com/authorities/name/>
            PREFIX madsrdf: <http://www.loc.gov/mads/rdf/v1#>
            PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
            PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
        INSERT DATA { 
        GRAPH name:n80002329 {
        madsrdf:fullerName [ a madsrdf:PersonalName ;
            rdfs:label "MACHADÃO de ASSIS" ] ;
                }
        }


In [3]:
fuseki_update = FusekiUpdate('http://localhost:3030', 'authorities')
response = fuseki_update.run_sparql(variant)
response.convert()

QueryBadFormed: QueryBadFormed: A bad request has been sent to the endpoint: probably the SPARQL query is badly formed. 

Response:
b'Encountered " "[" "[ "" at line 7, column 28.\r\nWas expecting one of:\r\n    <IRIref> ...\r\n    <PNAME_NS> ...\r\n    <PNAME_LN> ...\r\n    <VAR1> ...\r\n    <VAR2> ...\r\n    "a" ...\r\n    \n'

In [4]:
obj = {
  "oldVariant": {
    "value": "Machado de Assis, Joaquín María",
    "date": "1839-1908",
    "type": "PersonalName"
  },
  "newVariant": {
"value": "MACHADÃO",
    "type": "PersonalName"
  }
}

request = EditVariant(**obj)
id = 'n80002329'
variant = addVariantPersonalName(id, request)
print(variant)

NameError: name 'addVariantPersonalName' is not defined

In [3]:
fuseki_update = FusekiUpdate('http://localhost:3030', 'authorities')
response = fuseki_update.run_sparql(variant)
response.convert()

{'statusCode': 200, 'message': 'Update succeeded'}

In [ ]:
id = 'n80002329'
variant = f"""PREFIX name: <https://bibliokeia.com/authorities/name/>
        PREFIX madsrdf: <http://www.loc.gov/mads/rdf/v1#>
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

        WITH name:{id}

            DELETE {{
            name:{id} madsrdf:hasVariant ?hasVariant .
    ?hasVariant madsrdf:variantLabel "{request.oldVariant.value}, {request.oldVariant.date}" .
    ?hasVariant madsrdf:elementList ?elementList .
    ?elementList rdf:first ?first .
    ?first madsrdf:elementValue "{request.oldVariant.value}" .
    ?elementList rdf:rest ?rest .
    ?rest rdf:first ?date .
    ?date madsrdf:elementValue "{request.oldVariant.date}"
             }}

        INSERT {{ 
         name:{id} madsrdf:hasVariant ?hasVariant .
    ?hasVariant madsrdf:variantLabel "{request.newVariant.value}, {request.newVariant.date}" .
    ?hasVariant madsrdf:elementList ?elementList .
    ?elementList rdf:first ?first .
    ?first madsrdf:elementValue "{request.newVariant.value}" .
    ?elementList rdf:rest ?rest .
    ?rest rdf:first ?date .
    ?date madsrdf:elementValue "{request.newVariant.date}"
        }}

        WHERE {{ 
        name:{id} madsrdf:hasVariant ?hasVariant .
    ?hasVariant madsrdf:variantLabel "{request.oldVariant.value}, {request.oldVariant.date}" .
    ?hasVariant madsrdf:elementList ?elementList .
    ?elementList rdf:first ?first .
    ?first madsrdf:elementValue "{request.oldVariant.value}" .
    ?elementList rdf:rest ?rest .
    ?rest rdf:first ?date .
    ?date madsrdf:elementValue "{request.oldVariant.date}"
        
         }}"""

fuseki_update = FusekiUpdate('http://localhost:3030', 'authorities')
response = fuseki_update.run_sparql(variant)
response.convert()
# print(variant)
        

In [ ]:
obj = {
  "id": "n80002329",
  "label": "Machado de Assis",
  "date": "1839-1908",
  "birthDate": "1839-06-21",
  "birthPlace": "Rio de Janeiro (Brazil)",
  "deathDate": "1908-09-29",
  "fullerName": "Joaquim Maria Machado",
  "hasCloseExternalAuthority": ["http://id.worldcat.org/fast/48407", "http://www.wikidata.org/entity/Q311145"],
  "hasExactExternalAuthority": ["http://viaf.org/viaf/sourceID/LC%7Cn++80002329#skos:Concept"],
  "hasVariant": [
    {
      "value": "Machado de Assis, Joaquín María",
      "type": "PersonalName",
      "date": "1839-1908"
    },
    {
      "value": "Mashado de Assiz, Zhoakin",
      "type": "PersonalName",
      "date": "1839-1908"
    },
    {
      "value": "Assis, Machado de",
      "type": "PersonalName",
      "date": "1839-1908"
    }
  ],
  "hasAffiliation": [
    {
    "organization": "http://id.loc.gov/authorities/names/n82059239",
    "affiliationStart": "1897",
    "affiliationEnd": "1908"
    }
  ],
  "occupation": [
    "http://id.loc.gov/authorities/subjects/sh85034154",
    "http://id.loc.gov/authorities/subjects/sh85092863"

  ]
}

request = PersonalName(**obj)
request

In [ ]:
AUT = {
    "value": "Joaquim Maria Machado de Assis",
    "type": "PersonalName"
}
request = Authority(**AUT)
request

In [ ]:
id = "n80002329"
updates = EditAuthorityPersonalName(id, request)
print(updates)

In [ ]:
fuseki_update = FusekiUpdate('http://localhost:3030', 'authorities')
for i in updates:
    response = fuseki_update.run_sparql(i)
    print(response.convert())

In [ ]:
elementValue = f"""PREFIX name: <https://bibliokeia.com/authorities/name/>
            PREFIX madsrdf: <http://www.loc.gov/mads/rdf/v1#>
            PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
            
            WITH name:{id}
            DELETE {{ name:{id} madsrdf:elementList ?elementList .
    ?elementList rdf:first ?o .
    ?o madsrdf:elementValue ?elementValue  }}
            INSERT {{  name:{id} madsrdf:elementList ?elementList .
    ?elementList rdf:first ?o .
    ?o madsrdf:elementValue "{request.value}" . }}
            WHERE {{ name:{id} madsrdf:elementList ?elementList .
    ?elementList rdf:first ?o .
    ?o madsrdf:elementValue ?elementValue }}"""

fuseki_update = FusekiUpdate('http://localhost:3030', 'authorities')
response = fuseki_update.run_sparql(elementValue)
response.convert()

In [ ]:
id = 'n80002329'
name = f"""PREFIX name: <https://bibliokeia.com/authorities/name/>
            PREFIX madsrdf: <http://www.loc.gov/mads/rdf/v1#>
            
            WITH name:{id}
            DELETE {{ name:{id} madsrdf:authoritativeLabel ?o }}
            INSERT {{ name:{id} madsrdf:authoritativeLabel { f'"{request.value}, {request.date}" ; ' if request.date else f'"{request.label}" ;' } }}
            WHERE {{ name:{id} madsrdf:authoritativeLabel ?o }}"""
print(name)
fuseki_update = FusekiUpdate('http://localhost:3030', 'authorities')
response = fuseki_update.run_sparql(name)
response.convert()

In [ ]:
solr = Solr('http://localhost:8983/solr/authorities/', timeout=10)
responseSolr = solr.add([doc], commit=True)
responseSolr

In [ ]:
personalGraph = MakePersonalGraph(request)
print(personalGraph)

In [ ]:
fuseki_update = FusekiUpdate('http://localhost:3030', 'authorities')
response = fuseki_update.run_sparql(personalGraph)
response.convert()

In [ ]:
def MakeVariant(hasVariant):
    
    variants = list()
    for i in hasVariant:
        variant = f"""[ a madsrdf:PersonalName,
                madsrdf:Variant ;
            madsrdf:elementList ( [ a madsrdf:FullNameElement ;
                        madsrdf:elementValue "{i.value}" ] 
                         { '[ a madsrdf:DateNameElement ; madsrdf:elementValue "{}" ] ) ;'.format(i.date) if i.date else ') ;' }           
            { ' madsrdf:variantLabel "{}, {}" ]'.format(i.value, i.date) if i.date else 'madsrdf:variantLabel "{i.value}" ]' }"""
        variants.append(variant)
    return variants

variants = MakeVariant(request.hasVariant)


In [ ]:
def MakeAffiliation(hasAffiliation):

    affiliations = list()
    for i in hasAffiliation:
        affiliation = f"""[ a madsrdf:Affiliation;
            { f'madsrdf:affiliationEnd "{i.affiliationEnd}" ;' if i.affiliationEnd else "" }
            { f'madsrdf:affiliationStart "{i.affiliationStart}" ;' if i.affiliationStart else "" }
            madsrdf:organization <{i.organization}> ] """
        affiliations.append(affiliation)

    return affiliations



In [ ]:
closeExternal = "madsrdf:hasCloseExternalAuthority {} ; \n"
exactExternal = "madsrdf:hasExactExternalAuthority {} ; \n"
hasVariant = "madsrdf:hasVariant  {}  ; \n"
birthPlace = """madsrdf:birthPlace [ a madsrdf:Geographic ;
            rdfs:label "{}" ] ;"""
hasAffiliation = "madsrdf:hasAffiliation {} ;"
occupation = "madsrdf:occupation {} ; \n"

if request.hasVariant:
        variants = MakeVariant(request.hasVariant)
if request.hasAffiliation:
        affiliations = MakeAffiliation(request.hasAffiliation)


template = f"""PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> 
PREFIX madsrdf: <http://www.loc.gov/mads/rdf/v1#> 
PREFIX owl: <http://www.w3.org/2002/07/owl#> 
PREFIX ri: <http://id.loc.gov/ontologies/RecordInfo#> 
PREFIX name: <https://bibliokeia.com/authorities/name/> 
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

INSERT DATA {{
GRAPH name:{request.id} {{
 name:{request.id} a madsrdf:Authority, madsrdf:PersonalName ;
        madsrdf:adminMetadata [ a ri:RecordInfo ; 
        ri:recordChangeDate "{request.recordChangeDate}"^^xsd:dateTime ;
        ri:recordContentSource <http://id.loc.gov/vocabulary/organizations/brmninpa> ;
        ri:recordStatus "new"^^xsd:string
        ] ; 
        madsrdf:authoritativeLabel "{request.label}" ;
        madsrdf:elementList ( [ a madsrdf:FullNameElement ;
                madsrdf:elementValue "{request.label}" ] 
                { '[ a madsrdf:DateNameElement ; madsrdf:elementValue "{}" ]'.format(request.date) if request.date else '' } ) ;
        madsrdf:fullerName [ a madsrdf:PersonalName ;
            rdfs:label "{request.fullerName}" ] ;
        { closeExternal.format(', '.join([ f'<{i}>' for i in request.hasCloseExternalAuthority])) if request.hasCloseExternalAuthority else '' }
        { exactExternal.format(', '.join([ f'<{i}>' for i in request.hasExactExternalAuthority])) if request.hasExactExternalAuthority else '' } 
        { hasVariant.format(', '.join(variants)) if request.hasVariant else '' } 
        { f'madsrdf:birthDate "{request.birthDate}" ;' if request.birthDate else '' } 
        { birthPlace.format(request.birthPlace) if request.birthPlace else '' } 
        { f'madsrdf:deathDate "{request.deathDate}" ;' if request.deathDate else '' } 
        { hasAffiliation.format(', '.join(affiliations)) if request.hasAffiliation else '' } 
        { occupation.format(', '.join([ f'<{i}>' for i in request.occupation])) if request.occupation else '' } 
}} }}
"""
print(template)

In [ ]:
fuseki_update = FusekiUpdate('http://localhost:3030', 'authorities')
response = fuseki_update.run_sparql(template)
response.convert()

In [ ]:
g = Graph()
g.parse("http://id.loc.gov/authorities/names/n80002329.madsrdf.rdf")
g.serialize('machado.nt')

In [ ]:
g = Graph()
r = httpx.get('http://localhost:3030/authorities?graph=https://bibliokeia.com/authorities/topic/sh85017405')
g.parse(r.content)
g.serialize("subject.nt")